# Optimizing the vectorized simulations #

Besides speed, the benefit of using the vectorized simulation based on the torch framework
is that we can track all operations that happen across a simulation. This allows us to find
the gradients with respect to any intermediate variable, and more importantly the gradient
of the yield the final copies of some complex with respect to initial parameters.

## An Introduction to Automatic Differentiation ##

Automatic differentiation is a powerful technique based on a simple principle of calculus,
the multivariable chain rule. Imagine we have function $ y = f(x, w)$ and we want to minimize $y$ by adjusting parameter $w$ we can
this by computing $\frac {\partial y} {\partial w}$. Automatic differentiation allows us to do this
for arbitrary function $f$ without needing to know $f$ in advance.

Whenever variable $u$ is involved in any operation, we create a new internal variable $v$ representing the result of
the operation, pointer back to $w$ and any other variables, and the function describing the partial $\frac {\partial v} {\partial u}$.
In this way we construct a directed acyclic "compuation graph", where $w$ and other input variable $x$ are leaves and the final optimization
target $y$ is the root.

We can now calculate the partial of $y$ with respect to each intermediate variable, and eventually $w$ by repeatedly
applying the chain rule of differentiation, $\frac {\partial y} {\partial u} = \frac {\partial y} {\partial v}  \frac {\partial v} {\partial u}$

## Applying to Our Use Case ##

We can easily write the simulator described in the `vectorized_network_simulation.ipynb` notebook as a function. Let's define:

- Simulator: $S$
- Vector of association constants: $\vec{k_{on}}$
- Vector of Reaction $\Delta G$ values: $\vec{G}$
- Vector of initial copy numbers: $\vec{C_o}$
- Matrix mapping species to reactions: $\textbf{M}$
- Simulation run time: $t$
- Final copies of complete complex: $Y$
- maximum possible yield: $min(\vec{C_o})$

The simulator itself can be expressed as:

$Y = S(\vec{k_{on}}, \vec{G}, \vec{C_o}, \textbf{M}, t)$

From the yield $Y$ we can construct the full loss term:

$L = \frac{-Y}{min(\vec{C_o})} + ReLU(-1*(k - min\_constant))$

Here the first term is the percent of the theoretical maximum possible complete complex yield,
and the second term is designed to prevent the on or off association constants from getting unreasonably small
or negative. The ReLU function is a piecewise function that will be a linear positive penalty when $k < min\_constant$,
and will equal $0$ otherwise. Minimizing the full objective $L$ w.r.t. $k_{on}$ will find values for $k_{on}$ that
maximize full complex yield while still being physically reasonable.

Automatic differentiation as described in the above section provide a method for doing this. We can run the simulation
until completion, all the while constructing the computation graph. We can then compute all the gradients, and then
step the parameter values in the direction of the gradient.

In other words, we compute $\frac{\partial L}{\partial \vec{k_{on}}}$ via automatic differentiation,
and then perform the update $k_{on} = k_{on} + \lambda \times \frac{\partial L}{\partial \vec{k_{on}}}$.

We preform a simulation then update the parameters in the direction of the gradient a set number of times
before stopping.

## Optimization Example With AP2 ##

In [6]:
# first we need to import the required modules
# make sure jupyter path is correct for loading local modules
import sys
sys.path.append("../../")
import pickle as pk
import numpy as np

import copy
from steric_free_simulator import Optimizer
from steric_free_simulator import ReactionNetwork

To begin, we need to generate the reaction network from the input files (see `reaction_network_building.ipynb`).
We can't directly resolve a vectorized network from an input file.

In [7]:
base_input = '../input_files/ap2.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()

['default_assoc', 1.0]
['A']
20.0
['M']
20.0
['B']
20.0
['S']
20.0
Parsing rule...
SPLIT_01:  ['A(a)+B(b)', 'A(a!1).B(a!1)']
['A', 'B', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['A(b)+M(a)', 'A(b!1).M(a!1)']
['A', 'M', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['A(c)+S(a)', 'A(c!1).S(a!1)']
['A', 'S', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['B(b)+M(b)', 'B(b!1).M(b!1)']
['B', 'M', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['M(c)+S(b)', 'M(c!1).S(b!1)']
['M', 'S', '']
GGGGGGGGGgg
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f12940dcc90>, 'copies': tensor([20.], dtype=torch.float64), 'subunits': 1})
Node-2 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f12940dcc90>, 'copies': tensor([20.], dtype=torch.float64), 'subunits': 1})
-----
{'A'}
{'A'}
set()
Steric hindrance detected
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f12940dcc90>, 'copies': tensor([20.], dtype=torch.float64), 'subunits': 1})
Node-2 :  (1, {'struct': <

We can feed the vanilla (networkx based) network into the optimizer and vectorized simulator, but it will be copied and
converted to a vectorized network. Let's ensure the network is set to the proper initial conditions and then initialize the
optimizer.

**Optimization and Simulation Parameters**:

- **sim_runtime**: time (in seconds) that the simulation will run for.
- **optim_iterations**: number of simulations to run, optimizing parameters at each one.
- **learning rate**: factor to multiply calculated gradients by.
- **device**: The hardware device to run simulations and optimizations on.

If the parameter is not specified at optimizer construction, it is given a default value.

In [8]:
rn.reset()
optim = Optimizer(reaction_network=rn,
                  sim_runtime=1,
                  optim_iterations=500,
                  learning_rate=.001,
                  device='cpu')

Using CPU
A
Reactant Sets:
M
Reactant Sets:
B
Reactant Sets:
S
Reactant Sets:
AM
Reactant Sets:
(0, 1)
AB
Reactant Sets:
(0, 2)
AS
Reactant Sets:
(0, 3)
BM
Reactant Sets:
(1, 2)
MS
Reactant Sets:
(1, 3)
ABM
Reactant Sets:
(1, 5)
(2, 4)
(0, 7)
AMS
Reactant Sets:
(3, 4)
(1, 6)
(8, 0)
ABS
Reactant Sets:
(2, 6)
(3, 5)
BMS
Reactant Sets:
(3, 7)
(8, 2)
ABMS
Reactant Sets:
(9, 3)
(0, 12)
(1, 11)
(10, 2)
Before:  tensor([[-1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0., -1., -1.,  0., -1.,  1.,  1.,  1., -0., -0., -0., -0., -0., -0.,
         -0., -0., -0., -0., -0., -0.,  1.,  1., -0.,  1.],
        [-1.,  0.,  0., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0., -1.,  0.,  1., -0., -0.,  1.,  1.,  1.,  1., -0., -0.,
         -0., -0., -0., -0., -0., -0., -0., -0.,  1., -0.],
        [ 0., -1.,  0., -1.,  0.,  0.,  0., -1., -1., -1., -1.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0., -0.,  1., -0.,  1., -0., -0., -0.,  1.,  1

Now we can run the optimization. After optimization, we write the new "optimized" parameters
into a copy of a reaction network so that we can inspect them if desired.

In [4]:
optim.optimize()
final_rn = copy.deepcopy(rn)
optim.rn.update_reaction_net(final_rn)

Reaction Parameters before optimization: 
[Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.], dtype=torch.float64, requires_grad=True)]
Optimizer State: <bound method Optimizer.state_dict of Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)>
Using CPU
Start of simulation: memory Used:  61.2
Next time:  tensor(inf, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 0 was 29.0%
current params: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.], dtype=torch.float64)
Grad:  tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  61.1
yield on sim iteration 1 was nan%
current params: tensor([nan, nan, nan, nan, nan, nan, nan,

Let's visulalize the reults. The Optimizer tracks observables across all simulation iterations,
and thus we can use its `plot_observables` function to plot the results of a single simulation

**Plot the first simulation, before any optimization:**

In [13]:
optim.plot_observable(iteration=0)

TypeError: plot_observable() missing 1 required positional argument: 'nodes_list'

**Plot the final simulation, after all optimization iterations:**


In [ ]:
optim.plot_observable(iteration=-1)


Plot the change in complex yield over time

In [ ]:
optim.plot_yield()


Clearly the optimizer was able to increase the total yield of the final complex at time 1 second.
The explanation for this will be left to the results notebook.
